In [1]:
import langchain
import requests
import json
import sys
import gradio as gr
from bs4 import BeautifulSoup
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.agents import initialize_agent, AgentType
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.tools import tool
from typing import *

In [2]:
system_message =f"""You are an AI assistant who can have a natural conversation and also help with booking flights.

## **STRICT RULES for Flight Booking (FOLLOW EXACTLY AS GIVEN BELOW)**:

1. **Ask for the source city.**
2. **Ask for the destination city (must be different from source).**
3. **Call the function `check_flight_availability`** ONLY with the user's destination.
   - If it returns an empty list, respond: `"No flights to that city."`
   - If it returns flights, LIST THEM **EXACTLY as received** in a numbered format:
     ```
     1. Airline: <airline>, Time: <time>, Price: <price>, Duration: <duration>
     2. Airline: <airline>, Time: <time>, Price: <price>, Duration: <duration>
     ```
   - **DO NOT** modify, reorder, or add extra details to the flight list.

4. **Wait for the user to pick a flight option by number. DO NOT assume or preselect a flight.**
5. **Ask for the passenger's first name, last name, and age.**
6. **Before calling `book_flight`, ENSURE all required parameters are available.**  
   - **If any parameters are missing, ASK the user for them.**
   - **DO NOT proceed with the function call if data is incomplete.**
7. **Call `book_flight`** ONLY when ALL details are available.
   - **Show the real seat number and booking details** returned by the function.

## **STRICT BEHAVIOR RULES:**
- **DO NOT** rephrase, modify, or add extra conversational elements when listing flights or asking for details.
- **DO NOT** generate fake flights, seat numbers, or booking details—use ONLY function responses.
- **ENSURE source and destination cities are different.** If they are the same, request a different destination.
- **DO NOT ask unnecessary questions or add your own conversational style. FOLLOW THE EXACT FORMAT.**
- **Generate a new ticket file** in the format:
- **If no flights are available for a city, respond: `"No flights found for that city."`**
- **If the user requests a summary of all tickets, call `generate_report` with NO arguments.**
"""


In [3]:
###############################################################################
# 2) Flight Availability with Price & Duration
###############################################################################
flight_availability = {
    "london": [
        {
            "airline": "AirlinesAI",
            "time": "10:00 AM",
            "price": "$799",
            "duration": "8 hours"
        },
        {
            "airline": "IndianAirlinesAI",
            "time": "3:00 PM",
            "price": "$899",
            "duration": "8 hours"
        },
        {
            "airline": "AmericanAirlinesAI",
            "time": "8:00 PM",
            "price": "$999",
            "duration": "8 hours"
        },
    ],
    "paris": [
        {
            "airline": "EuropeanAirlinesAI",
            "time": "11:00 AM",
            "price": "$399",
            "duration": "7 hours"
        },
        {
            "airline": "BudgetAirlines",
            "time": "6:00 PM",
            "price": "$2399",
            "duration": "7 hours"
        },
    ],
    "tokyo": [
        {
            "airline": "TokyoAirlinesAI",
            "time": "12:00 PM",
            "price": "$4000",
            "duration": "5 hours"
        },
        {
            "airline": "FastFly",
            "time": "7:00 PM",
            "price": "$1400",
            "duration": "5 hours"
        },
    ],
    "berlin": [
        {
            "airline": "BerlinAirlinesAI",
            "time": "9:00 AM",
            "price": "$499",
            "duration": "6 hours"
        },
        {
            "airline": "AmericanAirlinesAI",
            "time": "4:00 PM",
            "price": "$899",
            "duration": "6 hours"
        },
    ],
    "nagpur": [
        {
            "airline": "IndianAirlinesAI",
            "time": "8:00 AM",
            "price": "$1000",
            "duration": "10 hours"
        },
        {
            "airline": "JetAirlines",
            "time": "2:00 PM",
            "price": "$1500",
            "duration": "10 hours"
        },
        {
            "airline": "AirlinesAI",
            "time": "10:00 PM",
            "price": "$800",
            "duration": "10 hours"
        },
    ],
}


In [7]:
####Using Langchain tools @tool to create functions######
#####Here the doc strings are necessary as the provide the llm the function definition as in what does the function do#######
def generate_seat_numbers(seed_value):
    '''
    function to random generate seatnumber
    '''
    random.seed(seed_value)
    return [
        f"{random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')}{random.randint(1, 99):02}"
        for _ in range(5)
    ]
@tool
def check_flight_availability(destination_city:str)-> str:
    """
    This functions checks if the given destinnation is available in the above(flight_availability)
    if not available an empty dict is returned
    """
    print(f"[TOOL] check_flight_availability({destination_city})")
    city = destination_city.lower()
    return flight_availability.get(city, [])

@tool
def book_flight(source:str=None,destination:str=None,name:str=None,age:int=None)->str:
    """
    Books a flight with the given details. Input should be a comma-separated string: source,destination,airline,time,price,duration,seat,first_name,last_name,age
    """
    # Collect missing parameters
    missing_params = [param for param, value in locals().items() if value is None]

    if missing_params:
        return f"Before booking, please provide: {','.join(missing_params)}."

    if source.lower()!=destination.lower():
        return "Error: Source and destination cannot be same"
      # Convert option index from string to integer
    try:
        idx = int(option_index)
    except ValueError:
        return "Error: flight option number is not a valid integer."

    flights = check_flight_availability(destination)
    if not flights:
        return f"Error: No flights found for {destination.title()}."

    pick = idx - 1
    if pick < 0 or pick >= len(flights):
        return f"Error: Invalid flight option #{idx} for {destination.title()}."

    chosen_flight = flights[pick]
    airline   = chosen_flight["airline"]
    dep_time  = chosen_flight["time"]
    price     = chosen_flight["price"]
    duration  = chosen_flight["duration"]
      
    seat_list = generate_seat_numbers(hash(destination + airline + str(len(flight_bookings))))
    chosen_seat = seat_list[0]

    new_booking = {
        "source":      source.title(),
        "destination": destination.title(),
        "airline":     airline,
        "time":        dep_time,
        "price":       price,
        "duration":    duration,
        "seat":        chosen_seat,
        "first_name":  first_name.title(),
        "last_name":   last_name.title(),
        "age":         age,
    }
    flight_bookings.append(new_booking)

    booking_number  = len(flight_bookings)
    ticket_filename = generate_ticket_file(new_booking, booking_number)

    confirmation = (
        f"Booking #{booking_number} confirmed for {first_name.title()} {last_name.title()}. "
        f"Flight from {source.title()} to {destination.title()} on {airline} at {dep_time}. "
        f"Ticket saved to {ticket_filename}."
    )
    print(f"[TOOL] {confirmation}")
    return confirmation

@tool
def generate_report():
    """
    Summarize ALL tickets in a single file called summary_report.txt.
    """
    print(f"[TOOL] generate_report called.")

    report_content = "Flight Booking Summary Report\n"
    report_content += "=============================\n"

    if not flight_bookings:
        report_content += "No bookings found.\n"
    else:
        for i, booking in enumerate(flight_bookings, start=1):
            report_content += (
                f"Booking #   : {i}\n"
                f"Passenger   : {booking['first_name']} {booking['last_name']}, Age {booking['age']}\n"
                f"Source      : {booking['source']}\n"
                f"Destination : {booking['destination']}\n"
                f"Airline     : {booking['airline']}\n"
                f"Departure   : {booking['time']}\n"
                f"Price       : {booking['price']}\n"
                f"Duration    : {booking['duration']}\n"
                f"Seat Number : {booking['seat']}\n"
                "-------------------------\n"
            )

    filename = "summary_report.txt"
    with open(filename, "w") as f:
        f.write(report_content)

    msg = f"Summary report generated => {filename}"
    print(f"[TOOL] {msg}")
    return msg


In [9]:
# Function to handle tool calls
def handle_tool_calls(response,tools):
    '''
    When the llm response back and in the response we have tool_calls ,this imply that llm is looking forward to call our functions 
    tool_calls: has the below pattern
    tool_calls=[{'name': 'check_flight_availability', 'args': {'destination_city': 'Paris'}, 'id': '45a1788c-133e-46e6-a513-10d71c3f8e3c', 'type': 'tool_call'}]
    here the args are function parameters defined above 
    '''
    ###checking if tool_Calls is available in response from llm,that is tool_calls is required
    if response.tool_calls:
        for tool_call in response.tool_calls:
            tool_dict = {tool.name.lower(): tool for tool in tools}
            # Get the selected tool dynamically
            selected_tool = tool_dict[tool_call["name"].lower()]
            tool_msg = selected_tool.invoke(tool_call) 
        
        return tool_msg  # Return a list of ToolMessage objects

        #return results
    return None

In [ ]:
def chat(message, history):
    """
    Handles conversation using ChatOllama with tool calling support.
    """
    # Initialize the LLM with tool binding
    print(f"in the chat function ::{message}, {history}")
    llm = ChatOllama(model='llama3.2')
    ###providing the above custom functions####
    mytools=[check_flight_availability, book_flight, generate_report]
    ###the functions are to be bind to llm as per langchain#######
    llm=ChatOllama(model="llama3.2").bind_tools([check_flight_availability, book_flight, generate_report])

    messages = [SystemMessage(content=system_message)] + history + [HumanMessage(content=message)]
    
    try:
        # Invoke the model
        response = llm.invoke(messages)
        
        # Check if tool call was requested
        if response.tool_calls:
            for tool_call in response.tool_calls:
                print(f"[INFO] Tool call requested: {tool_call}")
                tool_response = handle_tool_calls(response,mytools)  # Handle tool execution
                #print(repr(tool_response))
                print(f"[INFO] Tool response: {tool_response}")
                '''
                here the tool response , is a ToolMesssage having below format
                ToolMessage(content='[{"airline": "EuropeanAirlinesAI", "time": "11:00 AM", "price": "$399", "duration": "7 hours"}, 
                {"airline": "BudgetAirlines", "time": "6:00 PM", "price": "$2399", "duration": "7 hours"}]', 
                name='check_flight_availability', tool_call_id='45a1788c-133e-46e6-a513-10d71c3f8e3c')
                when this is passed to llm the llm would provide the final answer to the user
                '''
                messages.append(tool_response)
                print(f"before llm invoke the messages is {messages}")
                response = llm.invoke(messages)
                print(f"The final response is {repr(response)}")
        
            return response.content
    
    except Exception as e:
        print(f"[ERROR] {e}")
        sys.exit(1)
        gr.close_all()
        return "I'm sorry, something went wrong while processing your request."


In [ ]:
###############################################################################
#  Launch Gradio
###############################################################################
gr.close_all()
gr.ChatInterface(fn=chat, type="messages").launch(debug=True) #debug=True can be removed is debugging not required